In [28]:
#import libraries
import geopandas as gpd
import ee
import geemap
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import ee


In [33]:
# Intialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-fajarramadhana505")

In [59]:
# -----------------------------------------------------------------------------
# CELL 1: Area of Interest and Time Range Setup
# -----------------------------------------------------------------------------
# The JavaScript 'Brunei' object is not a standard Earth Engine geometry.
# We'll create a placeholder for a sample region. You can replace this with
# the actual geometry you intend to use. Here, we are creating a rectangle
# around the approximate location of Brunei.
Brunei = ee.FeatureCollection('users/fajarramadhana505/Brunei/admin_country_buff_1km')

# Optional: if asset contains multiple features and you want a single geometry
brunei_geometry = Brunei.geometry()

# Example: buffer the AOI by 10 km
geom = brunei_geometry.buffer(100)

# Define the start and end years for the 10-year period
start_year_range = 2015  # The first year of the dNBR calculation
end_year_range = 2024    # The last year for the second image in the pair

Map = geemap.Map()
Map.addLayer(Brunei, {}, 'Brunei AOI')
# Center the map on your area of interest
Map.centerObject(brunei_geometry, 8)
Map

Map(center=[4.529054083122191, 114.76102305285698], controls=(WidgetControl(options=['position', 'transparent_…

In [37]:
# -----------------------------------------------------------------------------
# CELL 2: Data Collection and Cloud Masking Function
# -----------------------------------------------------------------------------
# Input Landsat collections
l9sr = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
l5sr = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
lsr = ee.ImageCollection(l9sr.merge(l8sr).merge(l5sr))

# Function to cloud mask from the Fmask band of Landsat SR data
def maskLsr(img):
    qa = img.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
        
    return img.updateMask(mask)

In [55]:
# -----------------------------------------------------------------------------
# CELL 3: Automated dNBR Calculation and Export Loop
# -----------------------------------------------------------------------------
# Loop through the years to calculate dNBR for each pair of consecutive years
for year in range(start_year_range, end_year_range):
    print(f"Processing dNBR for years {year} and {year + 1}...")

    # Calculate composite for the first year (e.g., 2015)
    startDate1 = ee.Date.fromYMD(year, 1, 1)
    endDate1 = ee.Date.fromYMD(year + 1, 1, 1)
    composite1 = lsr.filterDate(startDate1, endDate1) \
                    .map(maskLsr) \
                    .median() \
                    .multiply(0.0000275) \
                    .add(-0.2)
    
    # Calculate composite for the second year (e.g., 2016)
    startDate2 = ee.Date.fromYMD(year + 1, 1, 1)
    endDate2 = ee.Date.fromYMD(year + 2, 1, 1)
    composite2 = lsr.filterDate(startDate2, endDate2) \
                    .map(maskLsr) \
                    .median() \
                    .multiply(0.0000275) \
                    .add(-0.2)
    
    # Calculate pre-fire and post-fire NBR for the year pair
    BurntIndices1 = composite1.expression(
        "(NIR - SWIR2) / (NIR + SWIR2)",
        {
            'SWIR2': composite1.select("SR_B7"),
            'NIR': composite1.select("SR_B5"),
        })

    BurntIndices2 = composite2.expression(
        "(NIR - SWIR2) / (NIR + SWIR2)",
        {
            'SWIR2': composite2.select("SR_B7"),
            'NIR': composite2.select("SR_B5"),
        })

    # Calculate dNBR
    dNBR_unscaled = BurntIndices1.subtract(BurntIndices2)
    dNBR_scaled = dNBR_unscaled.multiply(1000).clip(brunei_geometry)

Processing dNBR for years 2015 and 2016...
Processing dNBR for years 2016 and 2017...
Processing dNBR for years 2017 and 2018...
Processing dNBR for years 2018 and 2019...
Processing dNBR for years 2019 and 2020...
Processing dNBR for years 2020 and 2021...
Processing dNBR for years 2021 and 2022...
Processing dNBR for years 2022 and 2023...
Processing dNBR for years 2023 and 2024...


In [57]:
# -----------------------------------------------------------------------------
# CELL 4: Visualize a Single Year's Results (Placed outside the loop for clarity)
# -----------------------------------------------------------------------------
# You should run the loop once and then use the last calculated dNBR_scaled variable
# to add it to the map.

# Visualization parameters for the composites
composite_vis = {
    'bands': ['SR_B5', 'SR_B6', 'SR_B7'],
    'min': 0,
    'max': [0.5, 0.3, 0.2],
}

# Visualization parameters for dNBR
dNBR_vis = {
    'min': -1000,
    'max': 1000,
    'palette': ['green', 'yellow', 'red'] # A simple palette for dNBR
}

# SLD style for classified dNBR
sld_intervals = """
<RasterSymbolizer>
  <ColorMap type="intervals" extended="false">
    <ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>
    <ColorMapEntry color="#7a8737" quantity="-250" label="-250"/>
    <ColorMapEntry color="#acbe4d" quantity="-100" label="-100"/>
    <ColorMapEntry color="#0ae042" quantity="100" label="100"/>
    <ColorMapEntry color="#fff70b" quantity="270" label="270"/>
    <ColorMapEntry color="#ffaf38" quantity="440" label="440"/>
    <ColorMapEntry color="#ff641b" quantity="660" label="660"/>
    <ColorMapEntry color="#a41fd6" quantity="2000" label="2000"/>
  </ColorMap>
</RasterSymbolizer>"""

# Add the final dNBR layer to the map
Map.addLayer(dNBR_scaled, dNBR_vis, 'Final dNBR Scaled')

# Add the classified dNBR layer using the SLD style
Map.addLayer(dNBR_scaled.sldStyle(sld_intervals), {}, 'dNBR Classified')

# Center the map on your area of interest
Map.centerObject(brunei_geometry, 8)

# Display the map in the notebook
Map

Map(bottom=64264.0, center=[4.529054083122191, 114.76102305285698], controls=(WidgetControl(options=['position…

In [52]:
# Export images to Drive
for year in range(start_year_range, end_year_range):    
    description_unscaled = f'dNBR_unscaledBrunei_{year}_{year+1}'
    task_unscaled = ee.batch.Export.image.toDrive(
        image=dNBR_unscaled,
        description=description_unscaled,
        folder='dNBR_unscaled',
        maxPixels=1e13,
        region=brunei_geometry,
        scale=30
    )
    task_unscaled.start()
    print(f"Export task for {description_unscaled} started: {task_unscaled.id}")

    description_scaled = f'dNBR_scaled_{year}_{year+1}'
    task_scaled = ee.batch.Export.image.toDrive(
        image=dNBR_scaled,
        description=description_scaled,
        folder='dNBR_scaled',
        maxPixels=1e13,
        region=brunei_geometry,
        scale=30
    )
    task_scaled.start()
    print(f"Export task for {description_scaled} started: {task_scaled.id}")

print("All dNBR tasks have been submitted.")

Export task for dNBR_unscaledBrunei_2015_2016 started: AWA7EN3Z43OMOVYYPQBEL7VJ
Export task for dNBR_scaled_2015_2016 started: O7QJ6QYXC4RKTXJP4BV3F4W4
Export task for dNBR_unscaledBrunei_2016_2017 started: UQ5V2NFC6M4VCOE74PW7JIJW
Export task for dNBR_scaled_2016_2017 started: PPKTRQ6VULET6QIU3ZS7UO5V
Export task for dNBR_unscaledBrunei_2017_2018 started: DVNPELRQMIETEN4CJOJB4QWW
Export task for dNBR_scaled_2017_2018 started: WDJLIGRPCLLJ5EDCCS7545L4
Export task for dNBR_unscaledBrunei_2018_2019 started: R4HXFK7YRK6IIIZNG23YNPGK
Export task for dNBR_scaled_2018_2019 started: RGYFVERXML54U5WB7AHIO67G
Export task for dNBR_unscaledBrunei_2019_2020 started: ZOSEURX4BSAKIVQ2H7BPFF3I
Export task for dNBR_scaled_2019_2020 started: YXMKLGWQPMNLOASTHIN7BNMW
Export task for dNBR_unscaledBrunei_2020_2021 started: W4G2OJJ4IHX3G33OLJVIHQT3
Export task for dNBR_scaled_2020_2021 started: HIICKF5MHAKV4AOJQTMDKY4H
Export task for dNBR_unscaledBrunei_2021_2022 started: LG4Z237QLFOJV7H6US5HDTCR
Export t